Requirements

In [ ]:
%pip install mwparserfromhell
%pip install lxml
%pip install pyspark
%pip install nltk
%pip install numba
%pip install pandas
%pip install matplotlib
%pip install scipy
%pip install scikit-learn
%pip install seaborn

Various imports

In [ ]:
import csv
import re
import os.path
import shutil
import pandas as pd
import glob
import mwparserfromhell
import nltk
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np

from nltk.corpus import stopwords
from lxml import etree
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, explode, split, regexp_replace, sum
from scipy.stats import norm
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.datasets import make_blobs

Download delle stopwords

In [ ]:
%%capture
nltk.download("stopwords")

----

Esempio sul funzionamento

In [ ]:
%%capture
def removeNestedParentheses(s):
    lines = s.split('\n')
    print(len(lines))
    ret = []
    for line in lines:
        curr_line = ''
        skip = 0
        for i in line:
            if i == '[':
                skip += 1
            elif i == ']'and skip > 0:
                skip -= 1
            elif skip == 0:
                curr_line += i
        ret.append(curr_line+"\n")
    return '\n'.join(ret)

text = '''
I has a template! {{foo|bar|baz|eggs=spam}} See it <nowiki>or no<!-- revealed --></nowiki>[[File:wiki.png|thumb|Wikipedia logo]]? [[fox]] and [[dog|puppy]] New York also has [[public transport|public transportation]].


New York is a state in the Northeastern United States and is the 27th-most extensive, fourth-most populous, and seventh-most densely populated U.S. state. New York is bordered by New Jersey and Pennsylvania to the south and Connecticut, Massachusetts, and Vermont to the east. The state has a maritime border in the Atlantic Ocean with Rhode Island, east of Long Island, as well as an international border with the Canadian provinces of Quebec to the north and Ontario to the west and north. The state of New York, with an estimated 19.8 million residents in 2015, is often referred to as New York State to distinguish it from New York City, the state's most populous city and its economic hub.
With an estimated population of 8.55 million in 2015, New York City is the most populous city in the United States and the premier gateway for legal immigration to the United States. The New York City Metropolitan Area is one of the most populous urban agglomerations in the world. New York City is a global city, exerting a significant impact upon commerce, finance, media, art, fashion, research, technology, education, and entertainment, its fast pace defining the term New York minute. The home of the United Nations Headquarters, New York City is an important center for international diplomacy and has been described as the cultural and financial capital of the world, as well as the world's most economically powerful city. New York City makes up over 40% of the population of New York State. Two-thirds of the state's population lives in the New York City Metropolitan Area, and nearly 40% live on Long Island. Both the state and New York City were named for the 17th century Duke of York, future King James II of England. The next four most populous cities in the state are Buffalo, Rochester, Yonkers, and Syracuse, while the state capital is Albany.
The earliest Europeans in New York were French colonists and Jesuit missionaries who arrived southward from settlements at Montreal for trade and proselytizing. New York had been inhabited by tribes of Algonquian and Iroquoian-speaking Native Americans for several hundred years by the time Dutch settlers moved into the region in the early 17th century. In 1609, the region was first claimed by Henry Hudson for the Dutch, who built Fort Nassau in 1614 at the confluence of the Hudson and Mohawk rivers, where the present-day capital of Albany later developed. The Dutch soon also settled New Amsterdam and parts of the Hudson Valley, establishing the colony of New Netherland, a multicultural community from its earliest days and a center of trade and immigration. The British annexed the colony from the Dutch in 1664. The borders of the British colony, the Province of New York, were similar to those of the present-day state.
Many landmarks in New York are well known to both international and domestic visitors, with New York State hosting four of the world's ten most-visited tourist attractions in 2013: Times Square, Central Park, Niagara Falls (shared with Ontario), and Grand Central Terminal. New York is home to the Statue of Liberty, a symbol of the United States and its ideals of freedom, democracy, and opportunity. In the 21st century, New York has emerged as a global node of creativity and entrepreneurship, social tolerance, and environmental sustainability. New York's higher education network comprises approximately 200 colleges and universities, including Columbia University, Cornell University, New York University, and Rockefeller University, which have been ranked among the top 35 in the world.


== History ==


=== 16th century ===
In 1524, Giovanni da Verrazzano, an Italian explorer in the service of the French crown, explored the Atlantic coast of North America between the Carolinas and Newfoundland, including New York Harbor and Narragansett Bay. On April 17, 1524 Verrazanno entered New York Bay, by way of the Strait now called the Narrows into the northern bay which he named Santa Margherita, in honour of the King of France's sister. Verrazzano described it as "a vast coastline with a deep delta in which every kind of ship could pass" and he adds: "that it extends inland for a league and opens up to form a beautiful lake. This vast sheet of water swarmed with native boats". He landed on the tip of Manhattan and perhaps on the furthest point of Long Island. Verrazanno's stay in this place was interrupted by a storm which pushed him north towards Martha's Vineyard.
In 1540 French traders from New France built a chateau on Castle Island, within present-day Albany; due to flooding, it was abandoned the next year. In 1614, the Dutch under the command of Hendrick Corstiaensen, rebuilt the French chateau, which they called Fort Nassau. Fort Nassau was the first Dutch settlement in North America, and was located along the Hudson River, also within present-day Albany. The small fort served as a trading post and warehouse. Located on the Hudson River flood plain, the rudimentary "fort" was washed away by flooding in 1617, and abandoned for good after Fort Orange (New Netherland) was built nearby in 1623.


=== 17th century ===

Henry Hudson's 1609 voyage marked the beginning of European involvement with the area. Sailing for the Dutch East India Company and looking for a passage to Asia, he entered the Upper New York Bay on September 11 of that year. Word of his findings encouraged Dutch merchants to explore the coast in search for profitable fur trading with local Native American tribes.
During the 17th century, Dutch trading posts established for the trade of pelts from the Lenape, Iroquois, and other tribes were founded in the colony of New Netherland. The first of these trading posts were Fort Nassau (1614, near present-day Albany); Fort Orange (1624, on the Hudson River just south of the current city of Albany and created to replace Fort Nassau), developing into settlement Beverwijck (1647), and into what became Albany; Fort Amsterdam (1625, to develop into the town New Amsterdam which is present-day New York City); and Esopus, (1653, now Kingston). The success of the patroonship of Rensselaerswyck (1630), which surrounded Albany and lasted until the mid-19th century, was also a key factor in the early success of the colony. The English captured the colony during the Second Anglo-Dutch War and governed it as the Province of New York. The city of New York was recaptured by the Dutch in 1673 during the Third Anglo-Dutch War (1672–1674) and renamed New Orange. It was returned to the English under the terms of the Treaty of Westminster a year later.


== References ==


== Further reading ==

French, John Homer (1860). Historical and statistical gazetteer of New York State. Syracuse, New York: R. Pearsall Smith. OCLC 224691273. (Full text via Google Books.)
New York State Historical Association (1940). New York: A Guide to the Empire State. New York City: Oxford University Press. ISBN 978-1-60354-031-5. OCLC 504264143. (Full text via Google Books.)


== External links ==
New York at DMOZ
 Geographic data related to New York at OpenStreetMap'''



section_title_re = re.compile("^=+\s+.*\s+=+$")
content = ''
skip = False

for l in text.splitlines():
    line = l.strip()
    if "= references =" in line.lower():
        skip = True  # replace with break if this is the last section
        break
    if "= further reading =" in line.lower():
        skip = True  # replace with break if this is the last section
        continue
    if section_title_re.match(line):
        skip = False
        continue
    if skip:
        continue
    content+=line+'\n'
    
""" for x in content:
    if x != "":
        print(x.lower()+"\n")
 """
 
withoutPipeLinks = (re.sub(r"\[\[[^|\]]*\|([^|\]]*)]]", r"\1", content, flags=re.S))
withOutComments= re.sub('<!--.*?-->', '', withoutPipeLinks, flags=re.S)
cleaned= re.sub('<.*?>', '', withOutComments, flags=re.S)
testoPulito = removeNestedParentheses(cleaned)
wikicode = mwparserfromhell.parse(testoPulito)
templates = wikicode.filter_templates()
text = " ".join([testo for testo in testoPulito.split() if testo not in templates])
print(text)

----

<h1>Processamento del testo</h1>

Metodo creato per pulire il file, mantenendo soltanto le parole, eliminando tutti i tag (parentesi quadrate, ad esempio) e tutti i simboli di punteggiatura.
Inoltre vado anche a cancellare i paragrafi ed i loro titoli. 

In [ ]:
def removeNestedParentheses(s):
    lines = s.split('\n')
    ret = []
    for line in lines:
        curr_line = ''
        skip = 0
        for i in line:
            if i == '[' or i == '{' :
                skip += 1
            elif (i == ']' or i == '}' ) and skip > 0:
                skip -= 1
            elif skip == 0:
                curr_line += i
        ret.append(curr_line+"\n")
    return '\n'.join(ret)

def cleanUp(text):
    section_title_re = re.compile("^=+\s+.*\s+=+$")
    content = ''
    skip = False

    for l in text.splitlines():
        line = l.strip()
        if "= references =" in line.lower():
            skip = True
            break
        if "= further reading =" in line.lower():
            skip = True 
            continue
        if section_title_re.match(line):
            skip = False
            continue
        if skip:
            continue
        content+=line+'\n'
    
    withoutPipeLinks = (re.sub(r"\[\[[^|\]]*\|([^|\]]*)]]", r"\1", content, flags=re.S))
    withOutComments= re.sub('<!--.*?-->', '', withoutPipeLinks, flags=re.S)
    cleaned= re.sub('<.*?>', '', withOutComments, flags=re.S)
    testoPulito = removeNestedParentheses(cleaned)
    wikicode = mwparserfromhell.parse(testoPulito)
    templates = wikicode.filter_templates()
    text = " ".join([testo for testo in testoPulito.split() if testo not in templates])
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    if text == "":
        return "NullTextFound"
    else:
        return text

Metodo destinato ad eliminare tutte le <i>stop words</i>.

In [ ]:
stop_words = set(stopwords.words("english"))
def remove_w(text):
    return ' '.join(w for w in text.split() if w not in stop_words)

Creazione di un file csv contenente i titoli della pagina e le parole contenute all'interno del file XML.
\
Tengo conto dei primi 1000000 elementi.

In [ ]:
fileWords = open("fileWords.csv", "w+")
fileTitles = open("fileTitles.csv", "w+")

wordsWriter = csv.writer(fileWords)
titlesWriter = csv.writer(fileTitles)

titlesWriter.writerow(["index", "title"])
wordsWriter.writerow(["index", "text"])
section_title_re = re.compile("^=+\s+.*\s+=+$")
skip = False

i = 0
j = 0
totalCount = 0
titles_list = []
words_list = []
file_xml= ''
if os.path.isfile("fileWiki.xml"):
    file_xml = "fileWiki.xml"
elif os.path.isfile("file.xml"):
    file_xml = "file.xml"
else:
    print("File non trovato")
tupla_titolo = ()
tupla_testo = ()
for event, elem in etree.iterparse(file_xml):
    text = ""
    title=""
    
    if('title' in str(elem.tag)):
        if(elem.text is not None):
            i = i+1
            title = elem.text.lower()
            titlesWriter.writerow([i, title])

    if('text' in str(elem.tag)):
        if(elem.text is not None):
            j = j+1
            text = remove_w(cleanUp(elem.text.lower()))
            wordsWriter.writerow([j, text])

    if j == 1000000 and i == 1000000:
        break
    elem.clear()
    totalCount += 1
    if totalCount > 1 and (totalCount % 1000) == 0:
            percents = (totalCount/1000000)*100
            print(f"Percents: {percents:.2f}%", end="\r")
            if percents == 100:
                break

print("-------------------------------------------DONE------------------------------------------------------")

---

<h1> Analisi dei dati </h1>

Inizializziamo spark

In [ ]:
spark = SparkSession.builder.appName("mySparkApp").config("spark.driver.memory", "4g").config("spark.driver.maxResultSize", "12g").getOrCreate()

<b> Carico i due file come DataFrame </b> <i> poi unendoli in un unico DF tenendo come chiave d'unione l'index</i>. 
\
Vado quindi a pulire quel che é stato generato (una cartella) mantenendo solo quel che ci serve a noi: il file csv con i dati.
\
Alla fine vado anche a cancellare fileWords.csv e fileTitles.csv in quanto non ci serviraranno piú e ci fará risparmiare spazio.
\
\
<i> Attenzione: se esiste ancora una cartella di nome [merged_file](./merged_file), essa viene cancellata per evitare conflitti o errori. </i>

In [ ]:
if os.path.isdir("merged_file"):
    shutil.rmtree("merged_file")

titles_df = spark.read.csv("fileTitles.csv", header=True, inferSchema=True)
text_df = spark.read.csv("fileWords.csv", header=True, inferSchema=True)

merged_df = titles_df.join(text_df, "index")

merged_df.coalesce(1).write.csv("merged_file", header=True)

for filename in os.listdir("merged_file"):
    if filename.endswith(".csv"):
        os.rename(os.path.join("merged_file", filename), os.path.join("", "merged.csv"))
        shutil.rmtree("merged_file")
        
os.remove("fileWords.csv")
os.remove("fileTitles.csv")

Ora bisogna mappare le parole (contando quante ce ne sono) presenti in ogni cella del file CSV. Tiene conto quindi del titolo, la parola che é presente in quella pagina e quante volte viene utilizzata in quel titolo.
\
Eliminiamo quindi il file [merged.csv](./merged.csv).

In [ ]:
df = spark.read.format("csv").option("header", "true").load("merged.csv")
df = df.select("title", lower("text").alias("text"))

words_df = df.select("title", explode(split(regexp_replace("text", r'\W+', ' '), ' ')).alias("word"))

word_counts = words_df.groupBy("title", "word").count().orderBy("count", ascending=False)
word_counts.write.csv("countedWordsDir", mode="overwrite", header=True)

os.remove("merged.csv")

Vado ad unire tutti i files CSV creati precedentemente eliminando quindi la cartella che li conteneva.
\
Eliminiamo poi la cartella.

In [ ]:
csv_files = glob.glob("countedWordsDir/*.csv")

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# unisco i dataframe in uno solo
merged_df = pd.concat(dfs)

merged_df.to_csv("countedWords.csv", index=False)

shutil.rmtree("countedWordsDir")

> <b>L’output deve contenere, per ogni parola, la lista di pagine di wikipedia
che contengono quella parola</b>

Quante volte sono state utilizzate le singole parole? Salviamo il risultato, poi, nel file [countedWordsFinalOutput.csv](./countedWordsFinalOutput.csv), unendo i file precedentemente generati separatamente.

In [ ]:
df = spark.read.format("csv").option("header", "true").load("countedWords.csv")

# raggruppa le righe in base alla colonna "word" e somma la colonna "count"
sum_df = df.groupBy("word").agg(sum("count").alias("total_count"))
sum_df = sum_df.withColumn("total_count", sum_df["total_count"].cast("int"))

sum_df.write.mode("overwrite").csv("paroleContate", header=True)

# unisco i file
csv_files = glob.glob("paroleContate/*.csv")

dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

merged_df = pd.concat(dfs)
merged_df.to_csv("countedWordsFinalOutput.csv", index=False)

shutil.rmtree("paroleContate")

Elimino la colonna che ci é inutile (la colonna count) salvando il risultato su un ultimo file finale, da analizzare. Elimino poi il file [countedWords.csv](./countedWords.csv) che non ci servirá a nulla.

In [ ]:
df = pd.read_csv('countedWords.csv')
df.drop('count', axis=1, inplace=True)
df.to_csv('finalFileToAnalyze.csv', index=False)
os.remove('countedWords.csv')

Leggiamo tutte le parole presenti nella colonna "word" del file CSV. Utilizziamo un set in quanto non vogliamo che ci siano parole ripetute. Poi lo trasformiamo in una lista per renderlo indicizzabile e la trascriviamo in un file csv in modo da non dover eseguire ogni volta questa cella.

In [ ]:
df = pd.read_csv('finalFileToAnalyze.csv')
word_list = []
for row in df['word']:
    words = re.findall(r'\w+', str(row).lower())
    word_list.extend(words)

word_list = list(set(word_list))

Ora controllo quali parole sono presenti nel file finale da controllare. Se le parole sono presenti, aggiungo il titolo ad una lista. Si creerá infine una riga in un file CSV con la parola e la lista di titoli.
\
Eliminiamo [finalFileToAnalyze.csv](./finalFileToAnalyze.csv) in quanto inutile. Dovremmo utilizzare risults.csv per le analisi.

In [ ]:
with open('finalFileToAnalyze.csv', newline='') as csv_file:
    reader = csv.DictReader(csv_file)

    # dizionario per mantenere le associazioni tra parole e titoli
    word_title_dict = {}

    for row in reader:
        title = row['title']
        word = row['word']

        if word not in word_title_dict:
            word_title_dict[word] = []

        word_title_dict[word].append(title)

with open('results.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # header
    writer.writerow(['word', 'titles'])

    for word in word_list:
        if word in word_title_dict:
            titles = word_title_dict[word]
        else:
            titles = []

        writer.writerow([word, titles])
os.remove("finalFileToAnalyze.csv")


---
<h1>Grafici e statistiche</h1>

Genero delle statistiche sui dati usando il file [countedWordsFinalOutput.csv](./countedWordsFinalOutput.csv).

In [ ]:
temp_df = pd.read_csv('countedWordsFinalOutput.csv', header=0)
ordered = temp_df.sort_values(by=['total_count'], ascending=False)
print(ordered.head(20).describe())


Grafico a barre che tiene conto del numero di titoli per ogni parola.

In [ ]:
data = pd.read_csv('countedWordsFinalOutput.csv')
data = data.sort_values(by=['total_count'], ascending=False)
data = data.head(10)
x = data['word']
y = data['total_count']

plt.bar(x, y)
plt.title('Parole piú utilizzate')
plt.xlabel('Parola')
plt.ylabel('Counter')
plt.show()

Creazione di un grafico che dimostra quale parola viene utilizzata in piú titoli.
\
Utilizza lo split per dividere le varie parole contenute nella colonna "titles" visto che sono divise da una virgola.

In [ ]:
df = pd.read_csv('results.csv')

# creazione di una nuova colonna "num_titles" che conta il numero di titoli in "titles"
df['num_titles'] = df['titles'].str.split(',').apply(lambda x: len(x))

df_sorted = df.sort_values('num_titles', ascending=False)

colori = ['red', 'blue', 'green', 'orange', 'purple']
df_words = []
df_counts = []
# stampa dei primi 10 record con le parole della stessa riga
for i in range(10):
    row = df_sorted.iloc[i]
    df_words.append(row['word'])
    df_counts.append(row['num_titles'])
    
x = df_words
y = df_counts
plt.bar(x, y, color = colori)
plt.title('Parole presenti in piú titoli')
plt.xlabel('Parola')
plt.ylabel('Counter')
plt.xticks(
    rotation=90, 
    horizontalalignment='right',
    fontweight='normal',
    fontsize='large'  
)

plt.show()

Creo un file csv contenente le parole e il numero di titoli (la dimensione delle liste di ogni parola).

In [ ]:
df = pd.read_csv('results.csv')

# crea una nuova colonna con la grandezza della lista
df['list_size'] = df['titles'].apply(lambda x: len(x.split(',')))

# crea un nuovo df con le colonne "word" e "list_size"
new_df = df[['word', 'list_size']].copy()

new_df.to_csv('titlesSize.csv', index=False)

df = pd.read_csv("titlesSize.csv")

# crea un nuovo df con il conteggio delle occorrenze per ogni valore di list_size
counts = df.groupby('list_size').size().reset_index(name='occurrences')

counts.to_csv("occurrences.csv", index=False)

Crea il grafico analizzando i dati contenuti nel file csv creato sopra.

In [ ]:
df_titles = []
df_occs = []

data = pd.read_csv('occurrences.csv')
data = data.head(50)
x = data['list_size']
y = data['occurrences']

plt.plot(x,y)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)

plt.xlabel('Titoli contenenti n parole')
plt.ylabel('Occorrenze')
plt.title('Numero di titoli che contengono n parole')

plt.show()

Grafico che tiene conto dei (n) titoli per singola parola. Se ad esempio, ci sono 5 parole che contengono 1 titolo, nell'asse x rappresenteremo il numero di parole per titolo. Nell'asse y invece il numero di titoli che contengono quelle n parole. Teniamo conto dei primi 5.

In [ ]:
df_titles = []
df_occs = []

data = pd.read_csv('occurrences.csv')
data = data.head(5)
x = data['list_size']
y = data['occurrences']

fig, plot = plt.subplots()

plot.plot(x, y)

# formatta l'asse x
plot.xaxis.set_major_locator(mtick.MaxNLocator(integer=True))

# formatta l'asse y
fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
plot.yaxis.set_major_formatter(tick)

plt.xlabel('Titoli contenenti n parole')
plt.ylabel('Occorrenze')
plt.title('Numero di titoli che contengono n parole (primi 5 valori)')
plt.show()
shutil.rmtree("occurrences.csv")